In [ ]:
from swagger_parser import SwaggerParser

parser = SwaggerParser(swagger_path='swagger.json')  # Init with file

# Get an example of dict for the definition Foo
parser.definitions_example.get('Pet')

In [2]:
import json

with open("swagger.json","r") as infile:
    swagger = json.load(infile)

In [3]:
from faker import Faker
from chance import chance
import random
fake = Faker()


In [200]:
property_generators = {
    "category_name": lambda: "foo",
    "name": lambda: fake.name(),
    "date-time": lambda: fake.date_time(),
    "username":lambda: chance.email(),
    "email": lambda: chance.email(),
    "id":lambda: fake.uuid4(),
    "quantity":lambda: random.randint(0,100),
    "int":lambda: random.randint(0,100),
    "integer":lambda: random.randint(0,100),
    "bool": lambda: chance.boolean(),
    "boolean": lambda: chance.boolean(),
    "firstName": lambda: fake.name().split(" ")[0],
    "lastName": lambda: fake.name().split(" ")[-1],
    "password": lambda: fake.password(),
    "phone": lambda: fake.phone_number(),
    "url": lambda: chance.url(),
    "photoUrls": lambda: chance.url(),
    "status": lambda: "dead" # this is malformed
}


def generate_from_schema(schema_string, name):
    return lambda: list(generate(schema_string.split("/")[-1], name))[0]

def generate_array_from_schema(schema_string, name):
    return lambda: [generate_from_schema(schema_string, name)() for x in range(random.randint(0,10))]

def generate_from_enum(enum):
    return lambda: random.choice(enum)

def generate_array_from_enum(enum):
    return lambda: [generate_from_enum(enum)() for x in range(random.randint(0,10))]

def generate_prop_from_name(name):
    return lambda: property_generators[name]() if name in property_generators else name + str(random.randint(0,10))

def generate_array_from_name(name):
    return lambda: [generate_prop_from_name(name)() for x in range(random.randint(0,10))]

def generate_prop_from_type(typ):
    return lambda:  property_generators[typ]() 

def generate_prop_from_format(fmt):
    return lambda:  property_generators[fmt]() 

def generate_array_from_type(typ):
    return lambda:  [property_generators[typ]()]

def transform_instance_from_schema_to_nested(generator, prop_name):
    generated = generator()
    keys = list(generated.keys())
    for k in keys:
        generated[prop_name + "_" + k] = generated[k]
        del generated[k]
    return generated
    
def get_parameter_generator_from_definition_property(prop_name, prop_val):
    if "$ref" in prop_val:
        generated = generate_from_schema(prop_val["$ref"], prop_name if prop_name != "body" else None)
        return lambda: transform_instance_from_schema_to_nested(generated, prop_name)
    elif prop_val["type"] == "array" and "enum" in prop_val["items"]:
        return generate_array_from_enum(prop_val["items"]["enum"])
    elif prop_val["type"] == "array" and "type" in prop_val["items"] and prop_val["items"]["type"] == "string":
        return generate_array_from_name(prop_name)
    elif prop_val["type"] == "array" and "type" in prop_val["items"] and prop_val["items"]["type"] == "integer":
        return generate_array_from_type(prop_val["items"]["type"])
    elif prop_val["type"] == "array" and "$ref" in prop_val["items"]:
        return generate_array_from_schema(prop_val["items"]["$ref"], prop_name)
    elif prop_val["type"] == "string" and "enum" in prop_val:
        return generate_from_enum(prop_val["enum"])
    elif prop_val["type"] == "string":
        if "format" in prop_val:
            return generate_prop_from_format(prop_val["format"])
        else:
            return generate_prop_from_name(prop_name)
    elif prop_val["type"] in ["int", "integer", "bool", "boolean"]:
        return generate_prop_from_type(prop_val["type"])
    else:
        print(prop_val)
        raise Exception(prop_name)

def get_command_parameter_generator_from_method_param(param):
    if "schema" in param:
        if "$ref" in param["schema"]:
            return generate_from_schema(param["schema"]["$ref"], param["name"] if param["name"] != "body" else None)
        elif param["schema"]["type"] == "array":
            return generate_array_from_schema(param["schema"]["items"]["$ref"], param["name"])
        else: 
            raise Exception()
    elif "$ref" in param:
        return generate_from_schema(param["$ref"], param["name"])
    elif param["type"] == "array" and "enum" in param["items"]:
        return generate_array_from_enum(param["items"]["enum"])
    elif param["type"] == "array" and param["items"]["type"] == "string":
        return generate_array_from_name(param["name"])
    elif param["type"] == "string" and "enum" in param:
        return generate_from_enum(param["enum"])
    elif param["type"] == "string":
        return generate_prop_from_name(param["name"])
    elif param["type"] == "int" or param["type"] == "integer":
        return generate_prop_from_type(param["type"])
    elif param["type"] == "file":
        return None
    else:
        raise Exception(param)
        
object_generators = {}

def get_generator_for_definition(definition_name):
    definition = swagger['definitions'][definition_name] 
    def object_generator(prefix=None):
        instance = {}       
        # iterate through properties 
        for prop_name in definition['properties']:
            prefixed_prop_name = prefix + "_" + prop_name if prefix is not None else prop_name
            val_gen = get_parameter_generator_from_definition_property(prefixed_prop_name, definition['properties'][prop_name])
            instance[prop_name] = val_gen() if val_gen is not None else None
        yield instance
    return object_generator
    
for definition_name in swagger["definitions"]:
    object_generators[definition_name] = get_generator_for_definition(definition_name)

def generate(obj_name, prefix):
    if obj_name not in object_generators:
        raise Exception("Object " + obj_name + " is unknown")
    return object_generators[obj_name](prefix)

commands = {}
for path_name in swagger["paths"]:
    path = swagger["paths"][path_name]
    for method in path:
        command = path[method]["summary"] if len(path[method]["summary"]) > 0 else path[method]["description"]
        for param in path[method]["parameters"]:
            if param["in"] == "header":
                continue
            commands[command] = (param["name"],get_command_parameter_generator_from_method_param(param))
commands

{'Add a new pet to the store': ('body',
  <function __main__.generate_from_schema.<locals>.<lambda>()>),
 'Update an existing pet': ('body',
  <function __main__.generate_from_schema.<locals>.<lambda>()>),
 'Finds Pets by status': ('status',
  <function __main__.generate_array_from_enum.<locals>.<lambda>()>),
 'Finds Pets by tags': ('tags',
  <function __main__.generate_array_from_name.<locals>.<lambda>()>),
 'Find pet by ID': ('petId',
  <function __main__.generate_prop_from_type.<locals>.<lambda>()>),
 'Updates a pet in the store with form data': ('status',
  <function __main__.generate_prop_from_name.<locals>.<lambda>()>),
 'Deletes a pet': ('petId',
  <function __main__.generate_prop_from_type.<locals>.<lambda>()>),
 'uploads an image': ('file', None),
 'Place an order for a pet': ('body',
  <function __main__.generate_from_schema.<locals>.<lambda>()>),
 'Find purchase order by ID': ('orderId',
  <function __main__.generate_prop_from_type.<locals>.<lambda>()>),
 'Delete purchase or

In [207]:
import re
from datetime import datetime
import time

def random_prop(obj, obj_name):
    key = random.choice(list(obj.keys()))
    
    return str( key + " " +  str(obj[key]))
    
def construct_sentence_from_generated_method_parameters(parameter_name, parameter_val, sentence):
    if type(parameter_val) is list:
        parameter_val = [p if p is not None else "" for p in parameter_val]
        pstring = ",".join(set([random_prop(pv, parameter_name) if type(pv) is dict else pv for pv in parameter_val]))
        sentence.append(parameter_name)
        sentence.append(" ")
        sentence.append(pstring)
        sentence.append(",")
    elif type(parameter_val) is str or type(parameter_val) is bool or type(parameter_val) is int:
        sentence.append(", ")
        sentence.append(parameter_name)
        sentence.append(" ")
        sentence.append(str(parameter_val))
    else:        
        if parameter_val is None:
            return sentence
        for k in parameter_val:
            if type(parameter_val[k]) is dict:
                sentence.append(random_prop(parameter_val[k], k))
            elif type(parameter_val[k]) is list:
                pstring = ",".join([pv["name"] if type(pv) is dict else pv for pv in parameter_val[k]])
                pstring = pstring[1:]
                sentence.append(k)
                sentence.append(" ")
                sentence.append(pstring)
            else:
                sentence.append(k)
                sentence.append(" ")
                sentence.append(str(parameter_val[k]))
            sentence.append(",")
    return sentence

for command in commands:
    # generate an utterance that contains all of the command and all of the parameters
    parameter_name = commands[command][0]
    parameter_val = commands[command][1]() if commands[command][1] is not None else None
    sentence = construct_sentence_from_generated_method_parameters(parameter_name, parameter_val, [command, " with "])    
    sentence = [re.sub("([A-Z]+)", " \g<1>", word).lower() for word in sentence]
    print("".join(sentence).replace("  "," "))

 add a new pet to the store with id 47,category_id 61,name debbie vasquez,photo urls awuf.edu/risil/tovtutlu/buwo/dohcufmo/nepdohu/hezazo,tofzeso.net/heso/bupalzo/sotdoc/dobno/garu,wibbil.co.uk/fuppof/wijegac/gonra/callo,jomdofne.net/nuhir/jumwit/kednevfi/tumluzel/juvciziz/zavwirkah,fohburlep.ru/popso/hawejmi/gawmicun/jirazsen/wekecwak/peludjo,mezajuz.net/fumejcu/nibtuv/botego/jowpacpug/seslimal,hucwuro.io/cuhbul/lubbu/hujsusup/hunveb/zasuli/bevdispu,donze.co.uk/jukec/rakgivel/fake/kavte/lurhe/zawizrag,tags ags_name10,tags_name9,tags_name6,tags_name9,tags_name10,status available,
 update an existing pet with id 4,category_name foo,name caleb gonzalez,photo urls ,tags ags_name7,tags_name2,tags_name8,tags_name3,tags_name3,tags_name4,tags_name2,tags_name3,tags_name9,tags_name3,status pending,
 finds pets by status with status available,pending,
 finds pets by tags with tags tags4,tags8,tags9,tags6,
 find pet by id with , pet id 88
 updates a pet in the store with form data with , status d

In [191]:
sentence

'Delete user with , username rojo@dedo.net'